In [1]:
import sys


import numpy as np
import os
import torch
from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D, Trainer1D
from denoising_diffusion_pytorch.utils import *
from diffusion1D.pipeline import diffusion1D
import pandas as pd
from deepchem.feat.smiles_tokenizer import SmilesTokenizer
from rdkit import Chem
from rdkit.Chem import Draw, AllChem, rdchem
import matplotlib.pyplot as plt
from IPython.display import Image, display
import sys
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


/home/ec2-user/anaconda3/envs/diffusion1D_github/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/ec2-user/anaconda3/envs/diffusion1D_github/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:

pipeline = diffusion1D()

# Data preprocessing
data_config = pipeline.get_default_data_config()
data_config.task = "conditional"
data_config.file_path = "./diffusion1D/htp_md.csv"
data_config.block_size = 64

print(data_config)
train_dataset, test_dataset, vocab_size = pipeline.data_preprocessing(data_config)

# Load the model
model_config = pipeline.get_default_model_config()
model_config.vocab_size = vocab_size
model_config.block_size = data_config.block_size
pipeline.load_model(model_config)

# Train
train_config = pipeline.get_default_train_config()
train_config.ckpts_path = "./diffusion1D/ckpts"
# train_config.pretrain = 100 ## Refer to model-100.pt (step = 10000)
print(train_config.device)
train_config.num_steps = 0
pipeline.train(train_config)


# Generate
generate_config = pipeline.get_default_generate_config()
generate_config.model_index = 10
assert generate_config.task == data_config.task
print(generate_config)

results = pipeline.generate(generate_config)

# Evaluate
print(pipeline.evaluate())

block_size: 64
train_test_split: (0.8, 0.2)
task: conditional
file_path: ./diffusion1D/htp_md.csv

                              mol_smiles  conductivity
0           NC(=O)CSCC(CO[Cu])OC(=O)[Au]             1
1       CCC(F)C(=O)NC(CO[Cu])COC(=O)[Au]             0
2            CCSCCN(CCN[Cu])CCOC(=O)[Au]             1
3          C#CCN(CCOCCO[Cu])CCOC(=O)[Au]             1
4          CCC(COC(=O)[Au])C(=O)NCCO[Cu]             0
...                                  ...           ...
11403  COCC(CNC(COC(=O)[Au])C(=O)O)O[Cu]             0
11404     CC(C)COCCNC(CN[Cu])CNC(=O)[Au]             1
11405         COCC(CNCCCO[Cu])OC(=O)[Au]             1
11406    CCNCCNC(=O)CC(CNC(=O)[Au])O[Cu]             0
11407  CCCC(OC(=O)[Au])C(=O)NOCC(C)N[Cu]             0

[11408 rows x 2 columns]
['[PAD]', '[CLS]', '[SEP]', 'C', '(', ')', 'O', '=', 'N', 'F', 'Cl', 'S', '[O-]', '#', '[N+]', 'P', '[Si]', '[Cu]', '[PH]', '[Ag]', '[Ac]', '[Au]']
[0, 12, 13, 16, 17, 18, 19, 22, 23, 27, 28, 34, 36, 38, 41, 45, 47,

0it [00:00, ?it/s]

training complete
model_index: 10
num_samples: 100
task: conditional



loading from version 1.6.4


sampling loop time step: 100%|██████████| 1000/1000 [00:21<00:00, 46.00it/s]


21
C=CC=(CCN[Cu])CC=CCOC(=O)[Au][Ac]
CCCOC(=O)[Au])C(=O)FCC(COCCO[Cu][Ag]
CC=C(=O)CCCC=C(=O)[Au])O[Cu][Ac]
CFCCCCCO[Cu])C(=)(COC(=O)[Au][Ac]
CCCC=[Cu])CCNCCOC(=O)[Au][Ag]
CCCCCO[Cu])C(=O)FCCCOC(=O)[Au][Ag]
O=C([Au]))CCSCCSCCO[Cu][Ag]
CCCCO[Cu])CCOC(=O)[Au][Ac]
CC(C(CCO[Cu])CCOCCNC(=O)[Au][Ac]
CC(C=[Cu])C(=O)NCCC(OC(=O)[Au][Ag]
CC(CCOC(=O)[Au])NC(=O)(CO[Cu][Ag]
CCCCCC((N[Cu])C(=O)NCCC=CCOC(=O)[Au][Ag]
O=C([Au]))CCClCCNCCO[Cu][Ag]
CC(C(C()[Cu])NC(=O)CC(C(C(=)OC(=O)[Au][Ac]
C#CCN(CCO[Cu])CCOC(=O)[Au][Ag]
COCCCCNCCCOC(=O)[Au])CCO[Cu][PH]
O=C([Au])NCCFCC=CCO[Cu][Ag]
C)CCO[Si])CClC(=O)(NO)CCOC(=O)[Au][Ac]
CCCCNCCCCOC(=O)[Au])CCCO[Cu][Ag]
CNCC=CCCCN=(COC(=O)[Au])O[Cu][Ag]
CC(CCNC(=O)CN(C(CO[Cu])OC(=O)[Au][Ag]
CC(CCO[Cu])NCCCCCCOC(=O)[Au][Ag]
O=C([Au])=CCC=(C([N+]CCCC((CCC(O))=[Cu][Ac]
O=C([Au])OCCC=O)CC=[Cu][Ag]
CCCC(NCCC=[Cu])CCOC(=O)[Au][Ag]
CCCO[Cu])C(OO)OCC(C(COC(=O)[Au][Ac]
CCOCCO[Cu])CCOC(=O)[Au][Ag]
CC=CC=(CC=[Cu])CCOC(=O)[Au][Ag]
CC(OO)(C=C(CO[Cu])COC(=O)[Au][Ag]
COCCCO[Cu])C(=O)NCC)C

O=C([Au])=CCOCCCCNCCC=[Cu][Ag]
CC(C=(CNCC(COC(=O)[Au])O[Cu][PH]
CCCCCCCOC(=O)[Au])NO(C)CCCC=[Cu][Ac]
CC(C(CC)O[Cu])=C(=O)CCC(OC(=O)[Au][Ac]
CC(COC(CC)(N[Si])C(=O)OCCOC(=O)[Au][Ac]
CCCCCCCO[Cu])OC(=O)[Au][Ag]
CCCCNCCCC)COC(=O)[Au])C(=O)=CC=[Cu][Au]
CCN)CCOC(=O)[Au])=C(=O)CCCCl[Cu][Ag]
CC)C(CO[Cu])CCCCO[PH][Ag]
CFCCCC(CCO[Cu])C(=O)CCOC(=O)[Au][Ag]
C=CC=C(=O)CCF[Cu])CC=C(=O)[Au][Ag]
CNCC=CC(COC(=O)[Au])O[Cu][Ac]
O=C([Au])OCCOCCC=[Cu][Ag]
COCCNCCCO[Cu])C(=O)NCOC(=O)[Au][Ag]
CCCCCC=CCO[Cu])CC=C(=O)[Au][Ac]
CCCCCOC(=O)[Au])FCCCO[Cu][Ac]
C=CCC(CNCC(COC(=O)[Au]))[Cu][PH]
COCCCCO[Cu])F(CCC)C(=C(=O)COC(=O)[Au][Ac]
C[O-]CCCCCCOCCCO[Cu])NC(=O)[Au][Ag]
C=CCCC=CCCO(CO[Cu])C(C(OC(=O)[Au][Ac]
O=C([Au])OCCCCCCCCCO[Cu][Ag]
CCCF(C(=O)O(CC)C)C(O[Cu])OC(=O)[Au][Ag]
CCC(CNC(P(C(=O)N(C)CCCOC(=O)[Au][Ag]
CS(COCCCN(CCO[Cu])CCOC(=O)[Au][Ag]
CCOCC(OC(=O)[Au])C(O)(=(C(C)F[Cu][Ac]
CCC(=OOC()P)ClC(=O)C(C(OC(=O)[Au][Ag]
CC(CCOC(=O)[Au])CFCCO[Cu][Ac]
CCCC)COC(=O)[Au])C(=O)NCC=[Cu][Ag]
CCC[SEP]CCCCCC=CO[Cu])OC(=O)[Au]